In [ ]:
import os
import requests
from IPython.display import Audio, display, Markdown
from tkinter import Tk, filedialog
from dotenv import load_dotenv

# Load environment variables
load_dotenv()

# 1. Function to transcribe audio using your backend
def transcribe_audio(audio_file_path):
    url = "http://localhost:5000/api/whisper/transcribe"
    
    with open(audio_file_path, 'rb') as file:
        files = {'audio': file}
        response = requests.post(url, files=files)
        
    if response.status_code == 200:
        print("✅ Audio transcribed successfully!")
        return response.json()['text']
    else:
        print(f"❌ Failed to transcribe audio: {response.status_code}")
        print(response.text)
        return None

# 2. Function to generate speech using your backend
def generate_speech(text, emotion="neutral"):
    url = "http://localhost:5000/api/tts/generate"
    
    payload = {
        "text": text,
        "emotion": emotion
    }
    
    response = requests.post(url, json=payload)
    
    if response.status_code == 200:
        print("✅ Speech generated successfully!")
        
        output_file = "generated_speech.mp3"
        with open(output_file, "wb") as f:
            f.write(response.content)
        
        display(Audio(output_file, autoplay=True))
        return output_file
    else:
        print(f"❌ Failed to generate speech: {response.status_code}")
        print(response.text)
        return None

# 3. Function to save to history
def save_to_history(user_id, audio_file, transcript):
    url = "http://localhost:5000/api/history/save"
    
    data = {
        "userId": user_id,
        "original_audio": audio_file,
        "transcript": transcript
    }
    
    response = requests.post(url, json=data)
    
    if response.status_code == 200:
        print("✅ Transcript saved to history successfully!")
        return response.json()
    else:
        print(f"❌ Failed to save to history: {response.text}")
        return None

# Workflow steps
print("=== RehearseAI Workflow ===")
print("1. Transcribe Audio")
print("2. Generate Speech")
print("3. Save to History")
print("4. Complete Workflow (all steps)")

choice = input("Enter your choice (1-4): ")

if choice == "1":
    # Transcribe audio
    root = Tk()
    root.withdraw()
    audio_filename = filedialog.askopenfilename(title="Select an audio file")
    if audio_filename:
        transcript = transcribe_audio(audio_filename)
        if transcript:
            print("Transcript:", transcript)
            display(Markdown(transcript))

elif choice == "2":
    # Generate speech
    text = input("Enter text to convert to speech: ")
    print("\nAvailable emotions: neutral, happy, sad, angry, excited")
    emotion = input("Enter emotion (default: neutral): ") or "neutral"
    if text:
        generate_speech(text, emotion)

elif choice == "3":
    # Save to history
    user_id = input("Enter your user ID: ")
    audio_file = input("Enter audio filename: ")
    transcript = input("Enter transcript text: ")
    save_to_history(user_id, audio_file, transcript)

elif choice == "4":
    # Complete workflow
    root = Tk()
    root.withdraw()
    audio_filename = filedialog.askopenfilename(title="Select an audio file")
    if audio_filename:
        # Step 1: Transcribe
        transcript = transcribe_audio(audio_filename)
        if transcript:
            print("Transcript:", transcript)
            display(Markdown(transcript))
            
            # Step 2: Generate speech with emotion
            print("\nAvailable emotions: neutral, happy, sad, angry, excited")
            emotion = input("Enter emotion for TTS (default: neutral): ") or "neutral"
            audio_file = generate_speech(transcript, emotion)
            
            # Step 3: Save to history
            if audio_file:
                user_id = input("Enter your user ID to save to history: ")
                if user_id:
                    save_to_history(user_id, os.path.basename(audio_filename), transcript)

else:
    print("Invalid choice!")

=== RehearseAI Workflow ===
1. Transcribe Audio
2. Generate Speech
3. Save to History
4. Complete Workflow (all steps)
